In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [2]:
import sys, os, glob
import numpy as np
import pandas as pd
import seaborn as sns
import argparse, progressbar
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes

sns.set()

In [4]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor,AspectAwarePreprocessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet
##
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import backend as K
from imutils import paths

In [5]:
args = {
    "models": r"/content/drive/MyDrive/TrainingFolder/p6_ensemble/models"
}

In [6]:
(testx, testy) = cifar10.load_data()[1]
testx = testx.astype("float") / 255.0

# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
              "dog", "frog", "horse", "ship", "truck"]

# encode labels
lb = LabelBinarizer()
testy = lb.fit_transform(testy)

170500096/170498071 [==============================] - 11s 0us/step


In [7]:
model_paths = os.path.sep.join([args["models"], "*.model"])
model_paths = list(glob.glob(model_paths))

models = []

In [9]:
# loop over model paths and then add them to model list
for i, model_path in enumerate(model_paths):
    print(f"[INFO] loading model {i + 1}/{len(model_paths)}......")
    models.append(load_model(model_path))

[INFO] loading model 1/5......
[INFO] loading model 2/5......
[INFO] loading model 3/5......
[INFO] loading model 4/5......
[INFO] loading model 5/5......


In [11]:
# init the list of preds
print(f"[INFO] evaluating ensembles......")
preds = []

for model in models:
    preds.append(model.predict(testx, batch_size=64))

# avg all probs - this is the ensembling
predictions = np.average(preds, axis=0)
print(classification_report(testy.argmax(axis=1),
                            predictions.argmax(axis=1),
                            target_names=labelNames))

[INFO] evaluating ensembles......
              precision    recall  f1-score   support

    airplane       0.85      0.85      0.85      1000
  automobile       0.91      0.95      0.93      1000
        bird       0.83      0.67      0.74      1000
         cat       0.78      0.58      0.66      1000
        deer       0.78      0.78      0.78      1000
         dog       0.84      0.68      0.75      1000
        frog       0.63      0.97      0.76      1000
       horse       0.88      0.87      0.88      1000
        ship       0.92      0.91      0.92      1000
       truck       0.86      0.93      0.89      1000

    accuracy                           0.82     10000
   macro avg       0.83      0.82      0.82     10000
weighted avg       0.83      0.82      0.82     10000

